<a href="https://colab.research.google.com/github/Lursen/self-organizing-map/blob/main/Self_organizing_map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

All attributes except the identification number are used as they are given.
The only transformations are normalization to 0...1 for the continuous
input attributes and 1-of-6 encoding for the output attribute.

Attribute Information (no missing values):
0. Id number: 1 to 214
  0
1. RI: refractive index  1.51115...1.53393
  1
2. Na: Sodium  10.73...17.38 
  1
3. Mg: Magnesium  0.33...4.49
  1
4. Al: Aluminum  0.29...3.5
  1
5. Si: Silicon  69.81...75.41
  1
6. K: Potassium  0.02...6.21
  1
7. Ca: Calcium  5.43...16.19
  1
8. Ba: Barium  0.06...3.15
  1
9. Fe: Iron  0.01...0.51
  1
---------
  9 inputs

10. Type of glass: 1, 2, 3, 5, 6, or 7   (6 classes)
  6
---------
  6 outputs


In [ ]:
bool_in=0
real_in=9
bool_out=6
real_out=0
training_examples=107
validation_examples=54
test_examples=53

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
data = np.loadtxt('glass2.dt')
X = data[:, 0:9]
Y = data[:, 9:15]

y = []
for i in Y:
  y.append(list(i).index(1))

y = np.asarray(y)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.99, random_state=42)

test_data =  np.insert(X_train, 9, y_train, axis=1)

Iteration algorithm 

In [ ]:
def find_BMU(SOM,x):
    distSq = (np.square(SOM - x)).sum(axis=2)
    err = np.min(distSq, axis=None)
    g,h = np.unravel_index(np.argmin(distSq, axis=None), distSq.shape)
    return g,h,err
    
def update_weights(SOM, train_ex, learn_rate, radius_sq, 
                   BMU_coord, step=3):
    g, h = BMU_coord
    #if radius is close to zero then only BMU is changed
    if radius_sq < 1e-3:
        SOM[g,h,:] += learn_rate * (train_ex - SOM[g,h,:])
        return SOM
    # Change all cells in a small neighborhood of BMU
    for i in range(max(0, g-step), min(SOM.shape[0], g+step)):
        for j in range(max(0, h-step), min(SOM.shape[1], h+step)):
            dist_sq = np.square(i - g) + np.square(j - h)
            dist_func = np.exp(-(dist_sq*dist_sq)/ (2 *radius_sq * radius_sq))
            SOM[i,j,:] += learn_rate * dist_func * (train_ex - SOM[i,j,:])   
    return SOM    

def train_SOM(SOM, train_data, learn_rate = .1, radius_sq = 1, 
              lr_decay = .1, radius_decay = .1, epochs = 100):    
  learn_rate_0 = learn_rate
  radius_0 = radius_sq
  error = 1
  epoch = 0
  while ( (error > 0.03) & (epoch < epochs)):
      errors = []
      rand.shuffle(train_data)      
      for train_ex in train_data:
          g, h, err = find_BMU(SOM, train_ex)
          errors.append(err)
          SOM = update_weights(SOM, train_ex, 
                                learn_rate, radius_sq, (g,h))
      # Update learning rate and radius
      learn_rate = learn_rate_0 * np.exp(-epoch * lr_decay)
      radius_sq = radius_0 * np.exp(-epoch * radius_decay) 
      epoch +=1        
      error = np.mean(np.asarray(errors))  
      #print(error) 
  return SOM, epoch, error

def SOM_error(SOM, test_data):
  winners = []
  err = 0
  for test_ex in test_data:
    g,h, _ = find_BMU(SOM, test_ex)
    winners.append([g, h, test_ex[-1]])
    err = err + np.sqrt(np.sum(np.square(SOM[(g,h)] - test_ex)))
  return err, winners

In [ ]:
import time
# Dimensions of the SOM grid
m = 4
n = 4
input_n = 9
# Number of training examples
n_x = len(X_train)
rand = np.random.RandomState(0)
train_time = []
for i in range(10):
  # Initialize the SOM randomly
  SOM = np.random.ranf((m, n, input_n)).astype(float)
  # training
  max_epochs = 100
  time_start = time.time()
  SOM, epochs, _ = train_SOM(SOM, X_train, epochs=max_epochs)
  train_time.append(time.time() - time_start)
  err, winners = SOM_error(SOM, X_train)
  print(epochs, err)
print('mean time=', np.mean(np.asarray(train_time)))
print('std =', np.std(np.asarray(train_time)))

100 39.662849798005176
100 38.64539410455198
100 39.678346148646575
100 39.988260253052566
100 37.246899530327205
100 39.03273523114747
100 37.82331829303481
100 39.47417703669956
100 38.71690689634961
100 38.749687070147665
mean time= 3.6894821882247926
std = 0.13717785769261587


Batch algorithm

In [ ]:
def train_batch_SOM(SOM, train_data, radius_sq = 1, 
                    lr_decay = .1, radius_decay = .1, epochs = 10):    
  radius_sq = (SOM.shape[0] + SOM.shape[1]) // 2
  step = 3
  error = 1
  epoch = 0
  while ((error > 0.035) & (epoch < epochs)):
    batch_BMU = []
    errors = []
    winners = np.zeros((SOM.shape[0], SOM.shape[1], SOM.shape[2]))
    bmus_count = np.zeros((SOM.shape[0], SOM.shape[1], 1))
    for i in range(len(train_data)):
        g, h, err = find_BMU(SOM, train_data[i])
        winners[g, h] += train_data[i]
        bmus_count[g, h] += 1
        batch_BMU.append((g,h))
        errors.append(err)

    unique_BMU = np.unique(batch_BMU, axis=0)
    unique_BMU = tuple([tuple(row) for row in unique_BMU])
    avg = np.zeros_like(winners)
    for i in range(winners.shape[0]):
      for j in range(winners.shape[1]):
        if np.sum(winners[i][j]) != 0:
          avg[i,j] = winners[i][j] / bmus_count[i][j]
  
    # Update weights
    for BMU in unique_BMU:
      g,h = BMU
      num = 0
      denum = 0
      for i in range(max(0, g-step), min(SOM.shape[0], g+step)):
        for j in range(max(0, h-step), min(SOM.shape[1], h+step)):
          dist_sq = np.square(i - g) + np.square(j - h)
          dist_func = np.exp(-np.square(dist_sq) / (2 * np.square(radius_sq)))
          num += dist_func * avg[i,j,:] * bmus_count[i,j]
          denum += dist_func * bmus_count[i,j] 
        SOM[g,h,:] = num / denum
        
    radius_sq = radius_sq * np.exp(-epoch /100)
    error = np.mean(np.asarray(errors))  
    epoch += 1

  return SOM, epoch, error


In [ ]:
import time
# Dimensions of the SOM grid
m = 8
n = 8
input_n = 9
# Number of training examples
n_x = len(X_train)
rand = np.random.RandomState(0)
train_time = []
for i in range(10):
  # Initialize the SOM randomly
  SOM = np.random.ranf((m, n, input_n)).astype(float)
  # training
  max_epochs = 30
  time_start = time.time()
  SOM, epochs, _ = train_batch_SOM(SOM, X_train, epochs=max_epochs)
  train_time.append(time.time() - time_start)
  err, winners = SOM_error(SOM, X_train)
  print(epochs, err)
print('mean time=', np.mean(np.asarray(train_time)))
print('std =', np.std(np.asarray(train_time)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: RuntimeWarning: invalid value encountered in true_divide


25 28.640789537331052
26 28.938672155437413
26 28.31082870415984
30 30.264949393689786
30 30.960757178870846
28 29.498990796636146
29 30.464814415444863
27 28.891546386833664
27 26.60895935412031
28 27.34572587323391
mean time= 0.6566708803176879
std = 0.035862402733561886


Removing Dead Neurons

In [ ]:
# Dimensions of the SOM grid
m = 6
n = 6
input_n = 9
# Number of training examples
n_x = len(X_train)
rand = np.random.RandomState(0)
# Initialize the SOM randomly
SOM = np.random.ranf((m, n, input_n)).astype(float)
# training
max_epochs = 50
SOM, epochs, _ = train_SOM(SOM, X_train, epochs=max_epochs)
err, winners = SOM_error(SOM, X_train)
print(epochs, err)

50 29.826215027893063


In [ ]:
def SOM_labels(SOM, test_data):
  winners = np.empty((SOM.shape[0], SOM.shape[1]), dtype=object)
  for i in np.ndindex(winners.shape): winners[i] = []

  win_count = np.zeros((m,n))
  err = 0
  for test_ex in test_data:
    g,h, _ = find_BMU(SOM, test_ex[0:9])
    winners[g][h].append (test_ex[-1])
    win_count[int(g),int(h)] +=1
    err = err + np.sqrt(np.sum(np.square(SOM[(g,h)] - test_ex[0:9])))
  return win_count, winners

In [ ]:
win_count = np.zeros((m,n))
for test_ex in test_data:
  g,h, _ = find_BMU(SOM, test_ex[0:9])
  win_count[g,h] += 1 

new_SOM = []
for i in range(m):
  for j in range(n):
    if (win_count[i,j] < 3):
      SOM[i,j] = np.full(9,1000)

count, winners = SOM_labels(SOM,test_data)

In [ ]:
count

array([[ 8., 13.,  9.,  6.,  0.,  5.],
       [ 6.,  7.,  3.,  6.,  3.,  6.],
       [ 4., 23., 16.,  5.,  5.,  5.],
       [ 3., 15., 11.,  0.,  0., 14.],
       [12.,  9.,  0.,  0.,  0., 10.],
       [ 7.,  0.,  0.,  0.,  0.,  0.]])

Values from the biggest clusters

In [ ]:
a = winners[2,1]
b = winners[3,1]
c = winners[2,2]
unique, counts = np.unique(a, return_counts=True)
print (dict(zip(unique, counts)))
unique, counts = np.unique(b, return_counts=True)
print(dict(zip(unique, counts)))
unique, counts = np.unique(c, return_counts=True)
print(dict(zip(unique, counts)))

{0.0: 19, 1.0: 1, 2.0: 3}
{0.0: 6, 1.0: 6, 2.0: 3}
{0.0: 1, 1.0: 15}


In [ ]:
print(SOM[2,1])
print(SOM[3,1])
print(SOM[2,2])

[2.77536958e-01 3.17320505e-01 7.89585096e-01 3.01233743e-01
 6.02890232e-01 9.46647812e-02 2.88319237e-01 2.81163871e-11
 1.51150795e-06]
[2.72807778e-01 3.96893728e-01 8.13147862e-01 3.03678732e-01
 5.33672746e-01 9.04995801e-02 2.65735184e-01 4.89386648e-06
 8.38261981e-09]
[2.21646425e-01 3.54971943e-01 7.81469287e-01 3.80227700e-01
 6.10288234e-01 9.21994306e-02 2.40341615e-01 1.26226254e-06
 1.32816928e-05]
